<a href="https://colab.research.google.com/github/Edgar-La/hacking-civico/blob/master/proyecto/proyecto-hacking-civico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Titulo:__
En este documento encontras lo siguiente:

* Descripcion:

* Selección y obtención de los datos

* Exploración de los datos

* Visualización de los datos

* Conclusión 

* Referencias


## __Descripcion.__

## __Seleccion y obtencion de datos.__

## __Exploracion de los datos.__

## __Visualizacion de los datos.__

## __Conclusion.__

## __Referencias.__